In [ ]:
!pip install torch==1.11.0  # version recommended by source
!pip install git+https://github.com/gretelai/gretel-synthetics.git

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as md
from sklearn.preprocessing import MinMaxScaler
from pickle import dump, load
from skimage.util.shape import view_as_windows
import time

import torch

from gretel_synthetics.timeseries_dgan.dgan import DGAN
from gretel_synthetics.timeseries_dgan.config import DGANConfig, OutputType

In [ ]:
# set up DGAN config.

config = DGANConfig(
    max_sequence_len=100,
    sample_len=20,  # trying a larger sample_len
    batch_size=min(1000, 51),
    apply_feature_scaling=True,
    apply_example_scaling=False,
    use_attribute_discriminator=False,
    generator_learning_rate=1e-4,
    discriminator_learning_rate=1e-4,
    epochs=10000,
)

model = DGAN(config)

In [ ]:
# loading model for future use
model = model.load("../models/dgan_casing.pt")

In [ ]:
# Generate synthetic data - this ran near instantly
_, synthetic_features = model.generate_numpy(1000)

In [ ]:
def plot_100_slice(slice, ind):
    plt.plot(slice, label="pressure")
    ax = plt.gca()
    ax.title.set_text("Slice" + str(ind))
    plt.legend(prop={"size": 9})

In [ ]:
# plot random synthetic windows

figure = plt.figure(figsize=(10, 10))
figure.suptitle("Synthetic 100 row Window plots", fontsize=25, fontweight="roman")

for i in range(9):
    figure.add_subplot(3, 3, i + 1)
    index = np.random.choice(1000)
    sl = synthetic_features[index]
    plot_100_slice(sl, index)

plt.show()

In [ ]:
synthetic_features.shape

In [ ]:
reshaped_data = np.empty((100000, 2))

In [ ]:
init = 0
# outer loop around samples
for x in range(1000):
    # loop of each sample
    for y in range(100):
        reshaped_data[init][0] = x + 1000
        reshaped_data[init][1] = synthetic_features[x, y, 0]
        init += 1

In [ ]:
reshaped_data

In [ ]:
reshaped_df = pd.DataFrame(reshaped_data, columns=["id", "pressure"])

In [ ]:
reshaped_df

In [ ]:
reshaped_df.to_csv("../data/gasPump/synthetic_casing.csv", index=None)